# Задание
1. Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
2. Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
3. Попробуйте добавить +1 рекуррентный в encoder и decoder
4. Попробуйте заменить GRU ячейки на lstm-ячейки
5. Оцените качество во всех случаях

In [1]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
device

device(type='cuda')

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"([,\'])", r"", s)
    return s

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/MyDrive/HW/Deep learning/Data/rus.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[0:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

## Encoder RNN

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.n_layers*1, 1, self.hidden_size, device=device)

## Encoder_RNN_LSTM

In [8]:
class Encoder_RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(Encoder_RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers)

    def forward(self, input, hidden, cell):
        embedded = self.embedding(input).view(1, 1, -1)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        return output, (hidden, cell)

    def initHidden(self):
        return [torch.zeros(self.n_layers*1, 1, self.hidden_size, device=device),
                torch.zeros(self.n_layers*1, 1, self.hidden_size, device=device)]

## Decoder RNN

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=n_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

##Decoder_RNN_LSTM

In [10]:
class Decoder_RNN_LSTM(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(Decoder_RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        output = self.embedding(input).view(1, 1, -1)
        output, (hidden, cell) = self.lstm(output, (hidden, cell))
        output = self.softmax(self.out(output.squeeze(0)))
        return output, (hidden, cell)

    def initHidden(self):
        return [torch.zeros(self.n_layers*1, 1, self.hidden_size, device=device),
                torch.zeros(self.n_layers*1, 1, self.hidden_size, device=device)]

In [11]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [12]:
teacher_forcing_ratio = 0.5

def train(model_name, input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    if model_name == 'gru':
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
    else: # lstm
        for ei in range(input_length):
            encoder_output, (encoder_hidden[0], encoder_hidden[1]) = encoder(
                input_tensor[ei].unsqueeze(0), encoder_hidden[0], encoder_hidden[1])
            encoder_outputs[ei] = encoder_output.squeeze()

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if model_name == 'gru':
                decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            else:
                decoder_output, (decoder_hidden[0], decoder_hidden[1]) = decoder(
                decoder_input, decoder_hidden[0], decoder_hidden[1])

            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            if model_name == 'gru':
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
            else:
                decoder_output, (decoder_hidden[0], decoder_hidden[1]) = decoder(
                    decoder_input, decoder_hidden[0], decoder_hidden[1])
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [13]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [14]:
def trainIters(model_name, encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(model_name, input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [16]:
def evaluate(model_name, encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        if model_name == 'gru':
            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(
                    input_tensor[ei], encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]
        else:  # lstm
            for ei in range(input_length):
                encoder_output, (encoder_hidden[0], encoder_hidden[1]) = encoder(
                    input_tensor[ei].unsqueeze(0), encoder_hidden[0], encoder_hidden[1])
                encoder_outputs[ei] = encoder_output.squeeze()

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            if model_name == 'gru':
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
            else:
                decoder_output, (decoder_hidden[0], decoder_hidden[1]) = decoder(
                    decoder_input, decoder_hidden[0], decoder_hidden[1])

            topv, topi = decoder_output.data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [17]:
def evaluateRandomly(model_name, encoder, decoder, n=10):
    lucky = 0
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(model_name, encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)

        if output_sentence[0:len(pair[1])] == pair[1]:
            lucky += 1
        print('')

    print(f'Dullness: {lucky*100/n} %')

In [18]:
input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 444587 sentence pairs
Trimmed to 4185 sentence pairs
Counting words...
Counted words:
rus 3728
eng 2003
['он деиствует от своего имени .', 'he is acting on his own behalf .']


###  Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество

In [19]:
hidden_size = 256

encoder_1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters('gru', encoder_1, decoder_1, 75000, print_every=5000)
evaluateRandomly('gru', encoder_1, decoder_1)

0m 44s (- 10m 24s) (5000 6%) 3.0664
1m 24s (- 9m 10s) (10000 13%) 2.4733
2m 5s (- 8m 22s) (15000 20%) 1.9988
2m 45s (- 7m 35s) (20000 26%) 1.5631
3m 27s (- 6m 55s) (25000 33%) 1.2030
4m 8s (- 6m 12s) (30000 40%) 0.9450
4m 49s (- 5m 31s) (35000 46%) 0.6885
5m 30s (- 4m 49s) (40000 53%) 0.5307
6m 12s (- 4m 8s) (45000 60%) 0.3860
6m 53s (- 3m 26s) (50000 66%) 0.2687
7m 35s (- 2m 45s) (55000 73%) 0.1931
8m 16s (- 2m 4s) (60000 80%) 0.1346
8m 58s (- 1m 22s) (65000 86%) 0.0997
9m 39s (- 0m 41s) (70000 93%) 0.0764
10m 21s (- 0m 0s) (75000 100%) 0.0638
> он лучше всех подходит для этого проекта .
= he is the best for this project .
< he is the best for this project . <EOS>

> я больше не уставшии .
= i am no longer tired .
< i am no longer tired . <EOS>

> я убежден в твоеи невиновности .
= i am convinced of your innocence .
< i am convinced of your innocence . <EOS>

> я знаю об этом факте .
= i am aware of the fact .
< i am aware of the fact . <EOS>

> мы с ним сегодня увидимся .
= i am goin

### Попробуйте добавить +1 рекуррентный в encoder и decoder

In [20]:
hidden_size = 256

encoder_1 = EncoderRNN(input_lang.n_words, hidden_size, n_layers=2).to(device)
decoder_1 = DecoderRNN(hidden_size, output_lang.n_words, n_layers=2).to(device)
trainIters('gru', encoder_1, decoder_1, 75000, print_every=5000)
evaluateRandomly('gru', encoder_1, decoder_1)

0m 50s (- 11m 51s) (5000 6%) 3.0880
1m 38s (- 10m 42s) (10000 13%) 2.5703
2m 26s (- 9m 44s) (15000 20%) 2.1822
3m 14s (- 8m 54s) (20000 26%) 1.7649
4m 1s (- 8m 3s) (25000 33%) 1.4073
4m 50s (- 7m 16s) (30000 40%) 1.0905
5m 39s (- 6m 28s) (35000 46%) 0.8315
6m 28s (- 5m 39s) (40000 53%) 0.5976
7m 17s (- 4m 51s) (45000 60%) 0.4433
8m 5s (- 4m 2s) (50000 66%) 0.3057
8m 55s (- 3m 14s) (55000 73%) 0.2193
9m 43s (- 2m 25s) (60000 80%) 0.1524
10m 32s (- 1m 37s) (65000 86%) 0.1073
11m 23s (- 0m 48s) (70000 93%) 0.0940
12m 11s (- 0m 0s) (75000 100%) 0.0665
> он всего лишь ребенок .
= he is but a child .
< he is just a child . <EOS>

> она улыбнулась взмахнув рукои .
= she smiled waving her hand .
< she smiled waving her hand . <EOS>

> ты не наш друг .
= you are not our friend .
< you are not our friend . <EOS>

> ты мои лучшии друг .
= you are my best friend .
< you are my best friend . <EOS>

> он всегда занят как пчела .
= he is always as busy as a bee .
< he is always as busy as a bee . <EO

### Попробуйте заменить GRU ячейки на lstm-ячейки

In [21]:
hidden_size = 256

encoder_1 = Encoder_RNN_LSTM(input_lang.n_words, hidden_size).to(device)
decoder_1 = Decoder_RNN_LSTM(hidden_size, output_lang.n_words).to(device)
trainIters('lstm', encoder_1, decoder_1, 75000, print_every=5000)
evaluateRandomly('lstm', encoder_1, decoder_1)

0m 43s (- 10m 6s) (5000 6%) 3.2292
1m 22s (- 8m 56s) (10000 13%) 2.6760
2m 3s (- 8m 14s) (15000 20%) 2.4335
2m 43s (- 7m 30s) (20000 26%) 2.1165
3m 24s (- 6m 49s) (25000 33%) 1.7897
4m 5s (- 6m 8s) (30000 40%) 1.5141
4m 46s (- 5m 27s) (35000 46%) 1.2372
5m 27s (- 4m 46s) (40000 53%) 1.0232
6m 8s (- 4m 5s) (45000 60%) 0.7895
6m 50s (- 3m 25s) (50000 66%) 0.6315
7m 30s (- 2m 43s) (55000 73%) 0.4887
8m 12s (- 2m 3s) (60000 80%) 0.3780
8m 53s (- 1m 22s) (65000 86%) 0.2793
9m 35s (- 0m 41s) (70000 93%) 0.1908
10m 16s (- 0m 0s) (75000 100%) 0.1542
> я порядочныи человек .
= i am an honest person .
< i am an honest person . <EOS>

> я одинокии человек .
= i am a lonely man .
< i am a lonely man . <EOS>

> ты слишком молод чтобы путешествовать в одиночку .
= you are too young to travel alone .
< you are too young to travel alone . <EOS>

> он добр к окружающим .
= he is kind to those around him .
< he is kind to those around him . <EOS>

> она строга к ним .
= she is hard on them .
< she is ha

GRU на 2 слоях оказался хуже, чем LSTM